In [3]:
# store the configs and functions used in the batch testing 
import energyfuctions as ef
from sklearn.linear_model import RidgeCV
import pandas as pd
import numpy as np
config = [
   ('H', 'Stair1', 20),
   ('C', 'HVACType', '集中式全空气系统'),
   ('Y', 'Time', [2017,1,1])
]
config1=[0,1,2]

def combination_k(s, k):
    if k == 0: 
        return ['']
    subletters = []
    for i in range(len(s)):
        for letter in combination_k(s[i+1:], k-1):
            subletters += [str(s[i]) + str(letter)]
    return subletters

def fliter_data(train, coms):
    for con in coms:
        conf =  config[int(con)]
        train = do_filter_data(train, conf)
    return train

def do_filter_data(train, conf):
    if conf[0] == 'H':
        train = train[train[conf[1]] >= conf[2]]
    elif conf[0] == 'C':
        train = train[train[conf[1]] == conf[2]]
    elif conf[0] == 'Y':
        train = train[train[conf[1]] >= pd.Timestamp(conf[2][0], conf[2][1], conf[2][2])]
    return train

def collect_features(train):
    features =[]
    for col in train.columns:
        if col == 'meter/area' or col == 'target' or col == 'Record' or col == 'Time' or col == 'BuildingID':
            pass
        else:
            features.append(col)
    return features

In [4]:
# load the raw data and create the train the test data sets
train = pd.read_pickle('./data/df_all_noremoveC_addStair2_target_addMA.pkl')
train = ef.reduce_mem_usage(train)

building_number =  5 
# train['meter/area'] = train['Record']/train['Area']
train['target'] = np.log1p(train['Record']/train['Area'])
test = train[train['BuildingID'] == building_number]
test = test[test['Time'] >= '2017-01-01']
# train = train[train['BuildingID'] != building_number]
drop_cols = ['Type', 'HVACType', 'By_day_record', 'continous_value_flag', 'Stair1', 'Stair2']
# train.drop(drop_cols, 1, inplace=True)
test.drop(drop_cols, 1, inplace=True)

for i in range (1,4):
    combs = combination_k(config1, i)
    for com in combs:
        print(f'processing the train data with combine of {com}')
        train_set  = fliter_data(train, com)

        train_set.drop(drop_cols, 1, inplace=True)
        features = collect_features(train_set)
        print(f'training set size is {len(train_set)},  \n  test data set size is {len(test)}' )
        # output the best model path 
        test_preds = ef.do_RCV_train(train_set, test, features)

Mem. usage decreased to 129.65 Mb (18.7% reduction)
processing the train data with combine of 0
training set size is 682682,  
  test data set size is 17474


/Users/nanzhang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Meter 0, training rmse=0.58181, , test rmse=0.80151 
Meter 1, training rmse=1.45988, , test rmse=0.57371 
Overall training set rmse=0.86124
Overall test set for target building rmse=0.66233
processing the train data with combine of 1
training set size is 210056,  
  test data set size is 17474
Meter 0, training rmse=0.39608, , test rmse=0.27104 
Meter 1, training rmse=0.93992, , test rmse=0.56685 
Overall training set rmse=0.63771
Overall test set for target building rmse=0.51750
processing the train data with combine of 2
training set size is 344968,  
  test data set size is 17474
Meter 0, training rmse=0.83260, , test rmse=0.24905 
Meter 1, training rmse=1.39445, , test rmse=0.56577 
Overall training set rmse=1.03452
Overall test set for target building rmse=0.51338
processing the train data with combine of 01
training set size is 105028,  
  test data set size is 17474
Meter 0, training rmse=0.33190, , test rmse=0.27637 
Meter 1, training rmse=0.87067, , test rmse=0.56045 
Overall 